# 制作 PHT 的 memmap 文件

In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from gzhlaker import *
from tqdm import tqdm

## DIR Config

In [2]:
DATA_PATH = False
OUTPUT_PATH = True

In [3]:
if DATA_PATH:
    check_path("./data")

In [4]:
if OUTPUT_PATH:
    check_path("./output")

In [6]:
DATA_FILE_LIST = []

## 过程

In [12]:
TRAIN_PATH = "/sda/data/guozihang/PHOENIX-2014-T-release-v3_cache/train/"
TEST_PATH = "/sda/data/guozihang/PHOENIX-2014-T-release-v3_cache/test/"
DEV_PATH = "/sda/data/guozihang/PHOENIX-2014-T-release-v3_cache/dev/"

In [13]:
TRAIN_FILES = get_file_name(TRAIN_PATH)
TEST_FILES = get_file_name(TEST_PATH)
DEV_FILES = get_file_name(DEV_PATH)

In [16]:
def generate_info(FILES, PATH):
    info = []
    start = 0
    length = 0
    for index, i in tqdm(enumerate(FILES)):
        path = os.path.join(PATH, i)
        data = np.load(path) 
        length = data.shape[0]
        info.append({
            "path":path,
            "start":start,
            "end":start + length
        })
        start = start + length
    return info

In [19]:
train_info = generate_info(TRAIN_FILES, TRAIN_PATH)
test_info = generate_info(TEST_FILES, TEST_PATH)
dev_info = generate_info(DEV_FILES, DEV_PATH)

7096it [13:46,  8.58it/s]
642it [01:08,  9.36it/s]
519it [01:04,  8.03it/s]


In [26]:
with open("/sda/data/guozihang/phoenix2014-T-train.pickle", mode="wb") as f:
    pickle.dump(train_info, f)
with open("/sda/data/guozihang/phoenix2014-T-test.pickle", mode="wb") as f:
    pickle.dump(test_info, f)
with open("/sda/data/guozihang/phoenix2014-T-dev.pickle", mode="wb") as f:
    pickle.dump(dev_info, f)

In [22]:
def generate_memmap(info, path):
    T = info[-1]["end"]
    H, W, C = 256, 256, 3
    fp = np.memmap(path, dtype='uint8', mode='w+', shape=(T, H, W, C))
    for index, item in tqdm(enumerate(info)):
        path, start, end = item["path"], item["start"], item["end"]
        data = np.load(path)
        fp[start:end] = data[:]
        if index % 500 == 0:
            fp.flush()
    fp.flush()

In [27]:
generate_memmap(train_info, "/sda/data/guozihang/phoenix2014-T-bigarray-map-train")
generate_memmap(test_info, "/sda/data/guozihang/phoenix2014-T-bigarray-map-test")
generate_memmap(dev_info, "/sda/data/guozihang/phoenix2014-T-bigarray-map-dev")

7096it [24:10,  4.89it/s]
642it [02:10,  4.93it/s]
519it [01:57,  4.42it/s]


## OUTPUT OSS Config

In [ ]:
OUTPUT_FILE_LIST = [
    ["./output/csldaily.pickle", "240902B/output/csldaily.pickle", "240902B/readme.txt", "这个文件是一个列表文件，每个列表包含一个三个字段，分别是'path','start','end'用于指定数据的路径，数据在 memmap 文件的开始位置，数据在 memmap 文件的结束为止"]
]

In [ ]:
check_and_upload(OUTPUT_FILE_LIST)